# Notebook to Write Qualtrics Surveys

In [1]:
import pandas as pd
import numpy as np
import simple_search_func as ss
import re
import csv
import json

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# Load up the qsf!
#with open('Query_Template.qsf', 'r') as file:
with open('Query_Template.qsf', 'r') as file:
    data = json.load(file)

In [ ]:
choices_dict = {
    'love': ['hello', 'hi', 'are you', 'darling', 'sweetheart', 'honey', 'romance', 'affection', 'passion', 'devotion', 
             'adore', 'cherish', 'fondness', 'embrace', 'beloved', 'cuddle', 'valentine', 'kiss', 'heart', 'soulmate'],
    'sadness': ['meow', 'meow meow', 'meowwww', 'tears', 'gloom', 'despair', 'melancholy', 'sorrow', 'blue', 'grief', 
                'heartache', 'mourn', 'loneliness', 'misery', 'pain', 'regret', 'weep', 'hurt', 'lost', 'downcast'],
    'kiss': ['smoocher', 'kombucher', 'peck', 'smooch', 'liplock', 'mwah', 'pucker', 'osculation', 'romantic', 'passionate',
             'sweet', 'gentle', 'soft', 'warm', 'butterfly', 'cheek', 'forehead', 'eskimo', 'lovebite', 'chaste'],
    'happiness': ['joy', 'smile', 'laughter', 'cheer', 'delight', 'bliss', 'glee', 'ecstatic', 'merry', 'euphoria', 
                  'sunny', 'content', 'radiant', 'jubilant', 'bubbly', 'grateful', 'optimistic', 'carefree', 'vivacious', 'buoyant'],
    'anger': ['rage', 'fury', 'wrath', 'outrage', 'resentment', 'bitterness', 'spite', 'grudge', 'irritation', 'annoyance',
              'exasperation', 'hostility', 'hatred', 'indignation', 'fuming', 'seething', 'mad', 'infuriated', 'irate', 'tempest'],
    'fear': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless'],
    'fear2': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise2': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship2': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy2': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia2': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless']
}

In [ ]:
topics = list(choices_dict.keys())# will be dict keys
counter = 0
next_topic=None


# iterate
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    if x['Payload']['QuestionType']=='MC': # group rank, influences others
        text = topics[counter]
        old_text = x['SecondaryAttribute']
        new_q = f'Select whether or not the following quotes are similar to the query "{text}"'
        x['Payload']['QuestionDescription']= new_q
        x['SecondaryAttribute'] = new_q
        x['Payload']['QuestionText']= new_q
        choices = choices_dict[text]
        new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
        x['Payload']['Choices']=new_choices
        x['Payload']['ChoiceOrder']=list(new_choices.keys())
        counter +=1
counter = 0
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    if x['Payload']['QuestionType']=='RO': # rank only, depends on other
        # THIS COMES FIRST
        # update Secondary Attribute: replace whatever is in ""
        text = topics[counter]
        next_topic = text
        old_text = x['SecondaryAttribute']
        new_q = f'Rank these quotes on how funny they are!'
        x['SecondaryAttribute'] = new_q
        # update ['Payload']['QuestionText'] replace whatever is in ""
        x['Payload']['QuestionText']= new_q
        x['Payload']['QuestionDescription']= new_q
        counter+=1
        #counter +=1

# save
with open('test.qsf', 'w') as json_file:
    json.dump(data, json_file) 

In [21]:
# make this a function

def new_survey_file(choices_dict:dict,file_name:str,template_name:str='Query_Template.qsf'):
    with open(template_name, 'r') as file:
        data = json.load(file)
    topics = list(choices_dict.keys())# will be dict keys
    c_topics = topics.copy()
    # surveyid = 'newstring'

    # iterate
    for x in data['SurveyElements']:
        if type(x['Payload']) != dict:
            continue
        if 'QuestionType' not in x['Payload'].keys():
            continue
        # x['SurveyID']= surveyid
        if x['Payload']['QuestionType']=='MC': # group rank, influences others
            text = topics.pop()
            new_q = f'Select whether or not the following quotes are similar to the query "{text}"'
            x['Payload']['QuestionDescription']= new_q
            x['SecondaryAttribute'] = new_q
            x['Payload']['QuestionText']= new_q
            choices = choices_dict[text]
            new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
            x['Payload']['Choices']=new_choices
            x['Payload']['ChoiceOrder']=list(new_choices.keys())
        if x['Payload']['QuestionType']=='RO': # rank only, depends on other
            # update Secondary Attribute: replace whatever is in ""
            text = c_topics.pop()
            new_q = f'Rank these quotes on how funny they are!'
            x['SecondaryAttribute'] = new_q
            # update ['Payload']['QuestionText'] replace whatever is in ""
            x['Payload']['QuestionText']= new_q
            x['Payload']['QuestionDescription']= new_q

    # save
    with open(file_name, 'w') as json_file:
        json.dump(data, json_file) 

### Load up examples

Now that I have the function written, I need to load up the search engine.

In [22]:
# load up that search
complete = pd.read_csv('https://scmcqueen.github.io/StarTrekScriptData/complete_data.csv')
complete.columns = ['index','character', 'quote', 'scene', 'location', 'view',
       'episode', 'date', 'series', 'file']
complete['quote']=complete['quote'].apply(lambda text: " ".join(text.split()))
complete['character']=complete['character'].fillna('NA')
test_engine = ss.search_engine()
test_engine.bulk_load(complete[['quote']].to_dict()['quote'])
test_engine.add_df(complete)

We have added 0 documents. The engine now has 144211 documents.
Complete data added.


In [6]:
results=test_engine.bw_search('candle',30)
search_results = test_engine.pretty_print([x[0] for x in results])

In [7]:
indices = [x[0] for x in results]
choices = [' '.join(x) for x in search_results]

In [8]:
keywords_1 =  ['candle','dream','doctor','bajor','prophets','inequality','struggle','engineer','prune','humanity']

In [ ]:
indices = []
choices = []
search_term = []
file_input = {}

for term in keywords_1:
    # do the search
    results=test_engine.bw_search(term,20)
    search_results = test_engine.pretty_print([x[0] for x in results])
    # update values
    t_ind = [x[0] for x in results]
    t_choices = [' '.join(x) for x in search_results]
    t_search = [term]*len(t_ind)
    # update big list
    indices+=t_ind
    choices+=t_choices
    search_term+=t_search
    # update the dict
    file_input[term] = t_choices

new_survey_file(choices_dict=file_input,file_name='Evaluation/day_1.qsf',template_name='Query_Template.qsf')
pd.DataFrame({'indices':indices,'choices':choices,'search':search_term}).to_csv('Evaluation/day_1_data.csv')

In [24]:
keywords_1 =  ['candle','dream','doctor','bajor','prophets','inequality','struggle','engineer','prune','humanity',
               'story', 'religion', 'child', 'molly','counselor']
keywords_2 =  ['archaeology','chateau','trial','fiction','romance','wedding','father','kahless','chocolate','future',
               'bridge','saucer','coffee','romulans','colony']
keywords_3 =  ['raktajino','commit no errors','tanagra','borg','cardassia','poetry','spot','assimilate','unusual','barclay',
               'acquisition','rules','vedek','warp crystal',"bat'leth"]
keywords_4 =  ['blood wine','gagh','cook','celebrate','impossible','earl grey','jake','geordi','riker','poker',
               'phaser','occupation','conscious minds','tailor','neutral zone']
keywords_5 =  ['men','women','miles','worf','bashir','commander','captain','kira','troi','guinan',
               'shields','unification','mister spock','tribble','android']
keywords_6 =  ['unraveled','friendship','hailing frequencies','universe','traveler','wesley','good tea','good book','make it so','flute',
               'neutrino levels','noonian soong','breen','evidence','']
keywords_7 =  ['memory','merry man','obedient','honor','san francisco','new orleans','liberation','imagination','maquis','tribunal',
               'trill','lore','risa','detective','ezri']
keywords_8 =  ['not picard','ice cream','morn','weyoun','dukat','civilization','replicators','truth','lunch','past',
               'empath','betray','warp bubble','jadzia','pregnant']
keywords_9 =  ['vineyard','lwaxana','bucket','justice is justice','espionage','same lie','mother','klingon','baseball','moriarty',
               'vulcan','moon','lower decks','the academy','full impulse']
keywords_10 =  ['enterprise','ferengi','nagus','death','rootbeer','alpha quadrant','wormhole','tasha','holosuite','holmes',
                'the defiant','ensign ro','dabo girl','secret agent','oo-mox']

In [27]:
keywords = {#1:keywords_1,
            2:keywords_2,
            3:keywords_3,
            4:keywords_4,
            5:keywords_5,
            6:keywords_6,
            7:keywords_7,
            8:keywords_8,
            9:keywords_9,
            10:keywords_10
            }

for x in keywords.keys():
    counter = str(x)
    t_keys = keywords[x]

    indices = []
    choices = []
    search_term = []
    file_input = {}

    for term in t_keys:
        # do the search
        results=test_engine.bw_search(term,20)
        search_results = test_engine.pretty_print([x[0] for x in results])
        # update values
        t_ind = [x[0] for x in results]
        t_choices = [' '.join(x) for x in search_results]
        t_search = [term]*len(t_ind)
        # update big list
        indices+=t_ind
        choices+=t_choices
        search_term+=t_search
        # update the dict
        file_input[term] = t_choices

    new_survey_file(choices_dict=file_input,file_name=f'Evaluation/day_{counter}.qsf',template_name='Query_Template.qsf')
    pd.DataFrame({'indices':indices,'choices':choices,'search':search_term}).to_csv(f'Evaluation/day_{counter}_data.csv')